In [1]:
import src.pcg as pcg
pcg.init_obstacles_perlin((1,100,100), {})[0].shape

torch.Size([1, 100, 100])

In [281]:
import os
import torch

base_folder = "runs/"
stamp = "20231005-0443-1"
folder = os.path.join(base_folder, stamp)
data = []
for i in range(100):
    path = os.path.join(folder, f"{i}.pt")
    if os.path.exists(path):
       data.append(torch.load(path))

In [282]:
import importlib
import src.EINCASM as EINCASM
importlib.reload(EINCASM)

ein = EINCASM.EINCASM("config.yaml")
ein.sim.init_all_channels()
ein.sim.update()

TypeError: 'module' object is not callable

In [ ]:
ein.sim.update()
ein.sim.metadata


{'id': 'EINCASM Experiment', 'world_shape': (100, 100), 'period': 46.0}

In [378]:
import torch

channels = {
    't1': (torch.randn(1, 1, 1)**2).to(torch.int),
    't2': (torch.randn(2, 1, 1)**2).to(torch.int),
    't3': (torch.randn(3, 1, 1)**2).to(torch.int),
    'tsafe': (torch.randn(1, 1, 1)**2).to(torch.int),
}

def define_group(group_id, channel_ids, isgroup={
    't1': False,
    't2': False,
    't3': False,
    'tsafe': False,
    'Group_t1t2': True,
    'Group_t2t3': True,
    'Group_g1t1': True,
}):
    # Create a list of the selected tensors
    selected_tensors = [channels[ch_id] for ch_id in channel_ids]
    
    # Stack the tensors to create the group tensor
    group_tensor = torch.cat(selected_tensors, dim=0)    

    # Add the group tensor to the channels dictionary
    channels[group_id] = group_tensor
    
    # Update the channels dictionary with views from the group tensor
    start_idx = 0
    for ch_id in channel_ids:
        if isgroup[ch_id]:
            
        end_idx = start_idx + channels[ch_id].shape[0]
        channels[ch_id] = group_tensor[start_idx:end_idx].view(channels[ch_id].shape)
        start_idx = end_idx

# Test Case 1: Two groups sharing a channel
define_group('Group_t1t2', ['t1', 't2'])
define_group('Group_t2t3', ['t2', 't3'])

# Test Case 2: Groups of groups
define_group('Group_g1t1', ['Group_t1t2', 't1'])

channels['Group_g1t1'][-1, 0, 0] = 999
channels

{'t1': tensor([[[999]]], dtype=torch.int32),
 't2': tensor([[[0]],
 
         [[0]]], dtype=torch.int32),
 't3': tensor([[[0]],
 
         [[1]],
 
         [[0]]], dtype=torch.int32),
 'tsafe': tensor([[[0]]], dtype=torch.int32),
 'Group_t1t2': tensor([[[0]],
 
         [[0]],
 
         [[0]]], dtype=torch.int32),
 'Group_t2t3': tensor([[[0]],
 
         [[0]],
 
         [[0]],
 
         [[1]],
 
         [[0]]], dtype=torch.int32),
 'Group_g1t1': tensor([[[  0]],
 
         [[  0]],
 
         [[  0]],
 
         [[999]]], dtype=torch.int32)}

In [571]:
t1 = torch.randn(5, 2, 2)
a = t1[0]
b = t1[1]
c = t1[2]
d = t1[3]

g1 = t1[:2]
g2 = t1[1:3]

g1[0, 0, 0] = 999

a.data_ptr() == g1[0].data_ptr()


True

In [590]:
print(torch.mps.current_allocated_memory())
a = torch.randn(10, 1000, 1000, device='mps:0')
print(torch.mps.current_allocated_memory())
b = a[0]
c = a[1]
d = a[2]
print(torch.mps.current_allocated_memory())
del a, b, c, d
torch.mps.empty_cache()
print(torch.mps.current_allocated_memory())


22463488
62463488
62463488
22463488


In [36]:
import importlib
import src.Simulation as Simulation
importlib.reload(Simulation)
import src.Channel as Channel
importlib.reload(Channel)

sim = Simulation.Simulation(world_shape=(2,2))
sim.add_channel('t1', num_layers=3)

sim.add_subchannel('t1_1', 't1', indices=[0, 1])

sim.init_all_channels()

for ch in sim.channels.values():

AttributeError: 'Channel' object has no attribute 'data_ptr'

In [15]:
def f():
    return (1,2)

a,b = f()



In [4]:
[c.contents for c in sim.channels.values()]

AttributeError: 'Channel' object has no attribute 'contents'